### FE670 - Midterm Exam Part II

**Author**: Sid Bhatia

**Date**: November 2th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Dr. Steve Yang

#### Background

The S&P 500 stock market index, maintained by S&P Dow Jones Indices, comprises 500 large-cap American companies covering about 75 percent of the American equity market by capitalization. The index is weighted by market capitalization, so large companies account for relatively more of the index with the symbol SPY. The amount of change in the price of these stocks will be highly correlated, as they are all part of the larger market. Factor analysis will be used to reduce the dimensionality of the 500 stocks in the S&P 500 stock market index to reflect the influence of 11 major industry sectors.

After some research, you are given a set of 20 stocks within the S&P500 stock index to build a portfolio for your client. These 20 stocks are mixed with the top 10 most volatile and 10 least volatile during the period of 2013 and 2016. The ticker symbols of these 20 stocks are: ’CHK’, ’AMD’, ’LNT’, ’FCX’, ’VRTX’, ’WMB’, ’INCY’, ’NFLX’, ’MRO’, ’MU’, ’MCD’, ’BRK.B’, ’SO’, ’T’, ’KO’, ’PG’, ’JNJ’, ’WM’, ’RSG’, ’PEP’. You can get the daily stock prices and S&P 500 index prices for the same period from Yahoo finance using Python Yahoo finance package.

Your client is afraid of any exposure to the ENERGY sector due to the high uncertainty during the last 4 years. You were asked to perform a factor analysis using the 11 major industry sector ETFs as the benchmarks of the sector performances to reflect the influence of 11 major industry sectors. These factors can then be considered in your portfolio analysis for time period from January 01, 2017 to December 31, 2020.

Again you may use Python Yahoo finance package, use factor analysis on Vanguard Sector & specialty ETFs (’VOX’, ’VCR’, ’VDC’, ’VDE’, ’VFH’, ’VHT’, ’VIS’, ’VGT’, ’VAW’, ’VNQ’, ’VPU’) which represents the following corresponding sectors: ’COMM’, ’CONSUMER DISC’, ’CONSUMER ST’, ’ENERGY’, ’FINANCIALS’, ’HEALTH’, ’INDUSTRIALS’, ’TECHNOLOGY’, ’MATIREALS’, ’REAL ESTATE’, ’UTILITIES’. The data consists of the high, low, opening, and closing price of each of the 11 ETFs for each day. This includes all data from January 01, 2017 to December 31, 2020. We also have the S&P500 index prices during the same period.

Please use answer the following questions using the dataset:

#### Problem #1

Perform a factor analysis on returns of the 11 selected sector ETFs and the S&P 500 index from January 01, 2017 to December 31, 2020. Using a scree plot and by analyzing the eigenvalues of the correlation matrix, choose a sufficient number of factors for the market influence. Please provide the factor loadings for ‘VDE’ sector within the chosen important factors, and make your observations about the ENERGY sector influence. (10 points)